<a href="https://colab.research.google.com/github/AkuOO4/Agile-Blood-Management-Web-Application/blob/main/vectera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate
!pip install torch
!pip install transformers
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# Simple Way
from sentence_transformers import CrossEncoder

model = CrossEncoder('vectara/hallucination_evaluation_model')


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

from tqdm import tqdm
import pandas as pd


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('vectara/hallucination_evaluation_model')
tokenizer = AutoTokenizer.from_pretrained('vectara/hallucination_evaluation_model')


In [ ]:

def check_hall(pairs):
  # pairs = [context,output]
  inputs = tokenizer.batch_encode_plus(pairs, return_tensors='pt', padding=True)

  model.eval()
  with torch.no_grad():
      outputs = model(**inputs)
      logits = outputs.logits.cpu().detach().numpy()
      # convert logits to probabilities
      scores = 1 / (1 + np.exp(-logits)).flatten()

  # print(scores)
  return scores

In [ ]:
df = pd.read_csv('gemma_outp_cleaned.csv')
output = 'gemma_hall_score.csv'

# Initialize new columns for the outputs
# df['Lamma3_hall_score'] = None
df['OpenAI_hall_score'] = None
# df['ClaudeAI_hall_score'] = None
df['Gemma_hall_score'] = None

for i in tqdm(range(len(df)), desc="Processing"):
  # lamma3_outp = df.loc[i, 'Lamma3_output']
  # openai_outp = df.loc[i, 'OpenAI_Output']
  gemma_outp = df.loc[i, 'gemma_Output']

  context = df.loc[i, 'context']

  try:
    # print(type(context))
    # print("output: aopen_ai: ",type(openai_outp))
    # print("output: cluade_ai: ",type(claude_outp))

    df.at[i, 'Gemma_hall_score'] =check_hall([[context,gemma_outp]])
    # df.at[i, 'OpenAI_hall_score'] =check_hall([[context,openai_outp]])

    # df.at[i, 'ClaudeAI_hall_score'] = check_hall([[context,  claude_outp]])
    # Save the DataFrame to a CSV file after processing each row

  except Exception as e:
      df.to_csv(output, index=False)
      print(f"An error occurred while processing row {i}: {e}")

df.to_csv(output, index=False)
print("Processing complete and data saved.")

from google.colab import files
# Automatically download the file
files.download(output)
print("data download")


Processing: 100%|██████████| 294/294 [04:25<00:00,  1.11it/s]

Processing complete and data saved.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

data download


In [ ]:
from google.colab import runtime
runtime.unassign()